## Example Experiment ##
#### Dataset: Labeled Faces in the Wild ####
#### Experiment: (Experiment 3) Two party passive property inference attack ####
#### Details: ####
Training Task: Gender Classification <br />
Inference Property: Eyeglasses <br />
One of the workers is malicious and the other is the target of the attack

In [1]:
import simulator.server as server
import simulator.workerclass as worker
import simulator.workerhandler as wh
import topology_utils as tu

from collections import OrderedDict
import pandas as pd
import numpy as np
import copy
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm_notebook
%matplotlib inline

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchvision import transforms
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sattvik/envs/pytorch_env/lib/py

## Parameters ##

In [2]:
property_inf_epochs = 10
attack_epochs = 50
# epochs = 2
batch_size = 64
learning_rate = 0.001
server_learning_rate = 0.01
num_workers = 2
local_iterations = 2

inference_property = 'Sunglasses'
data_path = '../data/lfw/data/'
Path("./working_dir").mkdir(parents=True, exist_ok=True)
working_dir = "./working_dir"

active_steps = [2]

# default `log_dir` is "runs" - we'll be more specific here
!rm -rf ./runs/experiment_3
writer = SummaryWriter('runs/experiment_3')

## Divide Data between workers ##
10% of the data for testing <br />
25% of the data for training the property classifier <br />
5% of the data for validating the property classifier <br />
40% of the data for training <br />
20% of the data for validation

Special precaution is taken that images of the same person do not end up on multiple data subsets. We also check to ensure that the fraction of datapoints with the inference property is similar in all data subsets.

In [3]:
def split_data(df, percentage):
    all_names = df.person.unique()
    percentage_msk = np.random.rand(len(all_names)) < percentage
    percantage_names = all_names[percentage_msk]
    remaining_names = all_names[~percentage_msk]
    if (len(percantage_names) < 1) or (len(remaining_names)<1):
        raise Exception('Impossible split with lengths {} and {}'.format(len(percantage_names), len(remaining_names)))
    return df.loc[attributes_df['person'].isin(percantage_names)], df.loc[attributes_df['person'].isin(remaining_names)]

def get_property_percentage(df, inference_property):
    return len(df[(df[inference_property]>0)])/len(df)

attributes_df = pd.read_csv(data_path+'lfw_attributes.txt')

test_df, remaining_df = split_data(attributes_df, 0.1)
property_train_df, remaining_df = split_data(remaining_df, (0.25/0.9))
property_val_df, remaining_df = split_data(remaining_df, (0.05/0.65))
train_df, val_df = split_data(remaining_df, (0.4/0.6))

print(len(attributes_df), len(test_df), len(property_train_df), len(property_val_df), len(train_df), len(val_df), len(test_df) + len(property_train_df) + len(property_val_df) + len(train_df) + len(val_df))
print(get_property_percentage(attributes_df, inference_property),get_property_percentage(test_df, inference_property),get_property_percentage(property_train_df, inference_property),get_property_percentage(property_val_df, inference_property),get_property_percentage(train_df, inference_property),get_property_percentage(val_df, inference_property))
# print(attributes_df.keys())
# all_names = attributes_df.person.unique()
# tt_msk = np.random.rand(len(all_names)) < 0.8
# temp_train_names = all_names[tt_msk]
# test_names = all_names[~tt_msk]
# del all_names, tt_msk
# train_val_df = attributes_df.loc[attributes_df['person'].isin(temp_train_names)]
# test_df = attributes_df.loc[attributes_df['person'].isin(test_names)]

# # add column to indicate split
# train_val_df['target'] = 0
# # allocate half the people to the target
# names = train_val_df['person'].drop_duplicates()
# target_worker_names = names.sample(frac=1)[:int(len(names)/2)]
# target_worker_names = target_worker_names.reset_index(drop=True)

# # populate target field
# for index, row in train_val_df.iterrows():
#     if row['person'] in target_worker_names.values:
#         train_val_df['target'][index] = 1

# # print distribution of data
# print("entries with worker 1: {}, entries with worker 2: {}, entries in training set: {}, total entries: {}".format(sum(train_val_df['target']==1), sum(train_val_df['target']==0), len(test_df), len(attributes_df)))

13143 1867 3771 700 4503 2302 13143
0.014912881381724112 0.011783610069630423 0.01617608061522143 0.012857142857142857 0.014656895403064623 0.016507384882710686


## Define dataset class ##

In [4]:
class LFWDataset(Dataset):
    """LFW dataset."""

    def __init__(self, data_path, attributes_df, inference_property, transform=None):
        self.attributes_df = attributes_df
        self.data_path = data_path
        self.transform = transform
        self.inference_property = inference_property

    def __len__(self):
        return len(self.attributes_df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_path, "lfw_home/lfw_funneled", self.attributes_df.iloc[idx]['person'].replace(' ', '_'),"{}_{:04d}.jpg".format(self.attributes_df.iloc[idx]['person'].replace(' ', '_'),self.attributes_df.iloc[idx]['imagenum']))
#         img = torch.from_numpy(cv2.imread(img_path))
        img = Image.open(img_path, mode='r')
        
        label = self.attributes_df.iloc[idx]['Male']>0
        property_label = self.attributes_df.iloc[idx][self.inference_property]>0
        
        if self.transform:
            img = self.transform(img)

        return img, torch.tensor(label, dtype=torch.float), torch.tensor(property_label, dtype=torch.float)

## Training, Validation, and Evaluation functions ##

In [5]:
def perform_evaluation(val_model, dataloader):
    with torch.no_grad():
        epoch_loss = 0
        epoch_accuracy = 0
        for batch_idx, (data, target, inf_property) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
            # move data batch to GPU
            data = data.cuda()
            target = target.cuda()
            # forward pass
            output = val_model(data)
            loss = F.binary_cross_entropy(output, target.unsqueeze(1))
            # compute average loss an accuracy
            output = output.to('cpu')
            target = target.to('cpu')
            current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
            epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
            epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
    print("testing loss: {} and testing accuracy: {}".format(epoch_loss, epoch_accuracy))
    return epoch_loss, epoch_accuracy

In [6]:
def perform_validation(val_model, dataloader):
    with torch.no_grad():
        epoch_loss = 0
        epoch_accuracy = 0
        for batch_idx, (data, target, inf_property) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
            # move data batch to GPU
            data = data.cuda()
            target = target.cuda()
            # forward pass
            output = val_model(data)
#             print(output, target.unsqueeze(1))
            loss = F.binary_cross_entropy(output, target.unsqueeze(1))
            # compute average loss an accuracy
            output = output.to('cpu')
            target = target.to('cpu')
            current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
            epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
            epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
    print("val loss: {} and val accuracy: {}".format(epoch_loss, epoch_accuracy))
    return epoch_loss, epoch_accuracy

In [7]:
def perform_training(model, dataloader, optimizer, epoch=None, grads_dir=None):
    epoch_loss = 0
    epoch_accuracy = 0
    for batch_idx, (data, target, inf_property) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
        # move data batch to GPU
        data = data.cuda()
        target = target.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward pass
        output = model(data)
        loss = F.binary_cross_entropy(output, target.unsqueeze(1))
        # backward pass
        loss.backward()
        optimizer.step()
        # log gradients
        if grads_dir != None:
            grads = []
            for name, param in model.named_parameters():
                if param.grad is not None:
                    grads.append(param.grad.view(-1))
            grads = torch.cat(grads)
#             print(grads.shape, any(inf_property))
            torch.save(grads, '{}/grads_{}_{}_prop_{}.pt'.format(grads_dir, batch_idx, epoch, any(inf_property)))
        # compute average loss an accuracy
        output = output.to('cpu')
        target = target.to('cpu')
        current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
        epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
        epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
    print("train loss: {} and train accuracy: {}".format(epoch_loss, epoch_accuracy))
    return epoch_loss, epoch_accuracy

## Define Model ##

In [8]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.rnet = torch.hub.load('pytorch/vision:v0.5.0', 'resnet50', pretrained=True)
        self.rnet.fc = nn.Linear(2048, 1)
        
        for p in self.parameters():
            p.requires_grad = False

        for p in self.rnet.fc.parameters():
            p.requires_grad = True
        
        self.unfreeze_layer4()
        self.unfreeze_layer3()

    def forward(self, x):
        x = torch.sigmoid(self.rnet(x))
        return x
    
    def unfreeze_layer4(self):
        for p in self.rnet.layer4.parameters():
            p.requires_grad = True

    def unfreeze_layer3(self):
        for p in self.rnet.layer3.parameters():
            p.requires_grad = True

    def unfreeze_layer2(self):
        for p in self.rnet.layer2.parameters():
            p.requires_grad = True

    def unfreeze_layer1(self):
        for p in self.rnet.layer1.parameters():
            p.requires_grad = True

def define_model():
    return ResNet()

# Step1: Malicious node trains a gender classifier and generates a gradient dataset #

### Declare datasets and dataloaders ###

In [9]:
if 1 in active_steps:
    # datasets
    property_train_dataset = LFWDataset(data_path, property_train_df, inference_property, transform=transforms.Compose([
    #                                                                             transforms.RandomResizedCrop(224),
                                                                                transforms.RandomHorizontalFlip(),
                                                                                transforms.ToTensor()
                                                                                ]))
    property_val_dataset = LFWDataset(data_path, property_val_df, inference_property, transform=transforms.Compose([
    #                                                                         transforms.RandomResizedCrop(224),
                                                                            transforms.RandomHorizontalFlip(),
                                                                            transforms.ToTensor()]))

    # dataloaders
    property_train_dataloader = DataLoader(property_train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    property_val_dataloader = DataLoader(property_val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

    print(len(property_train_dataloader), len(property_val_dataloader))

### Sanity Check ###

In [10]:
if 1 in active_steps:
    for i , (img, label, property_inference) in enumerate(property_train_dataloader):
        print(property_inference, any(property_inference))
        for im, lab, prop in zip(img, label, property_inference):
            plt.imshow(np.moveaxis(np.asarray(im), 0, -1))
            plt.show()
            print(lab, prop)
            break
        break

### Define model and optimizer ###

In [11]:
if 1 in active_steps:
    property_model = define_model().cuda()
    property_optimizer = optim.Adam(property_model.parameters(), lr=learning_rate)

### Start Training ###
TODO: re-evaluate how to incorporate unfreezing schedule in this framework

In [12]:
if 1 in active_steps:
    # Clear working dir
    for filename in os.listdir(working_dir):
        file_path = os.path.join(working_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
    max_val_acc = 0
    for epoch in range(property_inf_epochs):
        # run train and val epochs
        print("Epoch: {}".format(epoch))
        property_model.train()
        train_loss, train_acc = perform_training(property_model, property_train_dataloader, property_optimizer, epoch=epoch, grads_dir=working_dir)
        writer.add_scalar('property_training loss', train_loss, epoch)
        writer.add_scalar('property_training accuracy', train_acc, epoch)
        property_model.eval()
        val_loss, val_acc = perform_validation(property_model, property_val_dataloader)
        writer.add_scalar('property_validation loss', val_loss, epoch)
        writer.add_scalar('property_validation accuracy', val_acc, epoch)
    writer.close()

# Step2: Malicious node trains a property inference classifier on the gradient dataset #

### Read and Split Gradient Dataset ###

In [13]:
if 2 in active_steps:
    # read_dataset
    grad_df = pd.DataFrame(columns=["grad_path", "label"])
    for filename in os.listdir(working_dir):
        file_path = os.path.join(working_dir, filename)
        if file_path.endswith('.pt'):
            entry = pd.DataFrame({"grad_path": [file_path],
                                 "label": [file_path[:-3].split('_')[-1]]})
            grad_df = grad_df.append(entry)
    grad_df = grad_df.reset_index(drop=True)
    
    # 80-20 split
    msk = np.random.rand(len(grad_df)) < 0.8
    attack_train_df = grad_df[msk]
    attack_val_df = grad_df[~msk]
    del grad_df
    print(len(attack_train_df), len(attack_val_df))

403 107


### Dataset Class ###

In [14]:
if 2 in active_steps:
    class GradientDataset(Dataset):
        """Face Landmarks dataset."""

        def __init__(self, gradients_df):
            self.gradients_df = gradients_df

        def __len__(self):
            return len(self.gradients_df)

        def __getitem__(self, idx):
            grad = torch.load(self.gradients_df.iloc[idx]['grad_path'], map_location=torch.device('cpu'))
            label = 1 if 'True' == self.gradients_df.iloc[idx]['label'] else 0

            return grad, torch.tensor(label, dtype=torch.float)
    # declare datasets
    attack_train_dataset = GradientDataset(attack_train_df)
    attack_val_dataset = GradientDataset(attack_val_df)
    # dataloaders
    attack_train_dataloader = DataLoader(attack_train_dataset, batch_size=8, shuffle=True, num_workers=num_workers)
    attack_val_dataloader = DataLoader(attack_val_dataset, batch_size=8, shuffle=True, num_workers=num_workers)

### Sanity Check ###

In [15]:
if 2 in active_steps:
    for i , (img, label) in enumerate(attack_train_dataloader):
        for grad, lab in zip(img, label):
            print(grad.shape, lab)
            break
        break

torch.Size([22065153]) tensor(0.)


### Define Attack Model ###

In [16]:
if 2 in active_steps:
    class AttackNet(nn.Module):
        def __init__(self):
            super(AttackNet, self).__init__()
            self.dropout_layer = nn.Dropout(p=0.2)
#             self.dense_layer = nn.Linear(22065153, 1)
            self.dense_layer = nn.Linear(7500000, 1)
#             self.dense_layer = nn.Linear(14565153, 1)

        def forward(self, x):
            op = torch.sigmoid(self.dense_layer(self.dropout_layer(x[:,14565153:])))
            return op

In [17]:
if 2 in active_steps:
    attack_network = AttackNet().cuda()
    print(attack_network)

AttackNet(
  (dropout_layer): Dropout(p=0.2, inplace=False)
  (dense_layer): Linear(in_features=7500000, out_features=1, bias=True)
)


### Define Optimizer and train \ val functions ###

In [18]:
if 2 in active_steps:
    attack_optimizer = optim.Adam(attack_network.parameters(), lr=0.005)

In [19]:
if 2 in active_steps:
    def perform_attack_validation(val_model, dataloader):
        with torch.no_grad():
            epoch_loss = 0
            epoch_accuracy = 0
            for batch_idx, (data, target) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
                # move data batch to GPU
                data = data.cuda()
                target = target.cuda()
                # forward pass
                output = val_model(data)
    #             print(output, target.unsqueeze(1))
                loss = F.binary_cross_entropy(output, target.unsqueeze(1))
                # compute average loss an accuracy
                output = output.to('cpu')
                target = target.to('cpu')
                current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
                epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
                epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
        print("val loss: {} and val accuracy: {}".format(epoch_loss, epoch_accuracy))
        return epoch_loss, epoch_accuracy

    def perform_attack_training(model, dataloader, optimizer):
        epoch_loss = 0
        epoch_accuracy = 0
        for batch_idx, (data, target) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
            # move data batch to GPU
            data = data.cuda()
            target = target.cuda()
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward pass
            output = model(data)
            loss = F.binary_cross_entropy(output, target.unsqueeze(1))
            # backward pass
            loss.backward()
            optimizer.step()
            # compute average loss an accuracy
            output = output.to('cpu')
            target = target.to('cpu')
            current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
            epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
            epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
        print("train loss: {} and train accuracy: {}".format(epoch_loss, epoch_accuracy))
        return epoch_loss, epoch_accuracy

### Train attack model ###

In [20]:
if 2 in active_steps:
    max_val_acc = 0
    for epoch in range(attack_epochs):
        # run train and val epochs
        print("Epoch: {}".format(epoch))
        attack_network.train()
        train_loss, train_acc = perform_attack_training(attack_network, attack_train_dataloader, attack_optimizer)
        writer.add_scalar('attack_training loss', train_loss, epoch)
        writer.add_scalar('attack_training accuracy', train_acc, epoch)
        attack_network.eval()
        val_loss, val_acc = perform_attack_validation(attack_network, attack_val_dataloader)
        writer.add_scalar('attack_validation loss', val_loss, epoch)
        writer.add_scalar('attack_validation accuracy', val_acc, epoch)
        if val_acc > max_val_acc:
            print("saving model on epoch {}".format(epoch))
            torch.save(attack_network.state_dict(), "models/exp3_best_attack_model.pt")
    writer.close()
    torch.save(attack_network.state_dict(), "models/exp3_final_attack_model.pt")

Epoch: 0


/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



train loss: 0.6914205948511759 and train accuracy: 0.5522875820889194


/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



val loss: 0.6937080110822406 and val accuracy: 0.6190476204667773
saving model on epoch 0
Epoch: 1


...


val loss: 0.8110545426607132 and val accuracy: 0.6190476204667773
saving model on epoch 47
Epoch: 48



train loss: 0.30150347305279157 and train accuracy: 0.8978758173830369



val loss: 0.8433228092534202 and val accuracy: 0.6220238102333886
saving model on epoch 48
Epoch: 49



train loss: 0.29286970402680196 and train accuracy: 0.8970588235294118



val loss: 0.8359351179429463 and val accuracy: 0.5952380959476743
saving model on epoch 49


## Exit here ##

In [21]:
raise SystemExit("Stop right there!")

SystemExit: Stop right there!

/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Define Model ##

In [ ]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.rnet = torch.hub.load('pytorch/vision:v0.5.0', 'resnet50', pretrained=True)
        self.rnet.fc = nn.Linear(2048, 1)
        
        for p in self.parameters():
            p.requires_grad = False

        for p in self.rnet.fc.parameters():
            p.requires_grad = True

    def forward(self, x):
        x = torch.sigmoid(self.rnet(x))
        return x
    
    def unfreeze_layer4(self):
        for p in self.rnet.layer4.parameters():
            p.requires_grad = True

    def unfreeze_layer3(self):
        for p in self.rnet.layer3.parameters():
            p.requires_grad = True

    def unfreeze_layer2(self):
        for p in self.rnet.layer2.parameters():
            p.requires_grad = True

    def unfreeze_layer1(self):
        for p in self.rnet.layer1.parameters():
            p.requires_grad = True

def define_model():
    return ResNet()

## Training, Validation, and Evaluation functions ##

In [ ]:
def perform_evaluation(val_model, dataloader):
    with torch.no_grad():
        epoch_loss = 0
        epoch_accuracy = 0
        for batch_idx, (data, target) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
            # move data batch to GPU
            data = data.cuda()
            target = target.cuda()
            # forward pass
            output = val_model(data)
            loss = F.binary_cross_entropy(output, target.unsqueeze(1))
            # compute average loss an accuracy
            output = output.to('cpu')
            target = target.to('cpu')
            current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
            epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
            epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
    print("testing loss: {} and testing accuracy: {}".format(epoch_loss, epoch_accuracy))
    return epoch_loss, epoch_accuracy

In [ ]:
def perform_validation(val_model, dataloader):
    with torch.no_grad():
        epoch_loss = 0
        epoch_accuracy = 0
        for batch_idx, (data, target) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
            # move data batch to GPU
            data = data.cuda()
            target = target.cuda()
            # forward pass
            output = val_model(data)
#             print(output, target.unsqueeze(1))
            loss = F.binary_cross_entropy(output, target.unsqueeze(1))
            # compute average loss an accuracy
            output = output.to('cpu')
            target = target.to('cpu')
            current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
            epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
            epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
    print("val loss: {} and val accuracy: {}".format(epoch_loss, epoch_accuracy))
    return epoch_loss, epoch_accuracy

In [ ]:
def perform_training(val_model, dataloader, optimizer):
    epoch_loss = 0
    epoch_accuracy = 0
    for batch_idx, (data, target) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
        # move data batch to GPU
        data = data.cuda()
        target = target.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward pass
        output = val_model(data)
        loss = F.binary_cross_entropy(output, target.unsqueeze(1))
        # backward pass
        loss.backward()
        optimizer.step()
        # compute average loss an accuracy
        output = output.to('cpu')
        target = target.to('cpu')
        current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
        epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
        epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
    print("train loss: {} and train accuracy: {}".format(epoch_loss, epoch_accuracy))
    return epoch_loss, epoch_accuracy

## Declare genuine worker ##

In [ ]:
class target_worker(worker.base_workerclass):
    def __init__(self, name, attributes_df, model):
        super().__init__(name, False)
        self.worker_attributes_df = attributes_df[attributes_df['target']==1]
        print("initializing genuine worker node with ",len(self.worker_attributes_df)," data points")
        self.model = model
        self.local_iters = local_iterations
        # train val split
        all_names = self.worker_attributes_df.person.unique()
        tt_msk = np.random.rand(len(all_names)) < 0.8
        train_names = all_names[tt_msk]
        val_names = all_names[~tt_msk]
        del all_names, tt_msk
        
        # set optimizer
        self.set_optim()
        # create train val and test dataframes
        train_df = self.worker_attributes_df.loc[self.worker_attributes_df['person'].isin(train_names)]
        val_df = self.worker_attributes_df.loc[self.worker_attributes_df['person'].isin(val_names)]
        
        train_dataset = LFWDataset(data_path, train_df, transform=transforms.Compose([
                                                    transforms.RandomResizedCrop(224),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.ToTensor()
                                                    ]))
        val_dataset = LFWDataset(data_path, val_df, transform=transforms.Compose([
                                                            transforms.RandomResizedCrop(224),
                                                            transforms.RandomHorizontalFlip(),
                                                            transforms.ToTensor()]))
        del train_df, val_df
        
        self.train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        self.val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        print(len(self.train_dataloader), len(self.val_dataloader))
    
    def set_param(self, w):
        self.model.load_state_dict(w)
    
    def get_params(self):
        return self.model.state_dict()
    
    def set_optim(self):
        self.optim = optim.Adam(self.model.parameters(), lr=learning_rate)
    
    def client_update(self, global_epoch):
        global writer
        self.model = self.model.cuda()
        prev_w = copy.deepcopy(self.model.state_dict())
        # unfreeze layers
        if 5 == global_epoch:
            self.model.unfreeze_layer3()
        if 20 == global_epoch:
            self.model.unfreeze_layer2()
        if 50 == global_epoch:
            self.model.unfreeze_layer1()
        for epoch in range(self.local_iters):
            # run train and val epochs
            print("sub-epoch: {}".format(epoch))
            self.model.train()
            train_loss, train_acc = perform_training(self.model, self.train_dataloader, self.optim)
            writer.add_scalar('training loss_'+self.name, train_loss, (global_epoch*self.local_iters)+epoch)
            writer.add_scalar('training accuracy_'+self.name, train_acc, (global_epoch*self.local_iters)+epoch)
            self.model.eval()
            val_loss, val_acc = perform_validation(self.model, self.val_dataloader)
            writer.add_scalar('validation loss_'+self.name, val_loss, (global_epoch*self.local_iters)+epoch)
            writer.add_scalar('validation accuracy_'+self.name, val_acc, (global_epoch*self.local_iters)+epoch)
        graddif = OrderedDict()
        for (item1, item2) in zip(self.model.state_dict().items(),prev_w.items()):
            key1=item1[0]
            value1=item1[1]
            key2=item2[0]
            value2=item2[1]
            diffval = value1-value2
            graddif.update({key1:diffval.cpu()})
        self.model = self.model.cpu()
        return graddif

## Declare malicious worker ##

In [ ]:
# class malicious_worker(worker.base_workerclass):
#     def __init__(self, attributes_df, model):
#         super().__init__(True)
#         self.worker_attributes_df = attributes_df[attributes_df['target']==0]
#         print("initializing malicious worker node with ",len(self.worker_attributes_df)," data points")
#         self.model = model
#         self.local_iters = 5
    
#     def set_param(self, w):
#         self.model.load_state_dict(w)
    
#     def set_optim(self):
#         self.optim = optim.Adam(self.model.parameters(), lr=learning_rate)
    
#     def client_update(self):
#         print('ss')

## Initialize components of our simulations ##

In [ ]:
server1 = server.server(server_learning_rate)
workers = wh.workerhandler([target_worker("w1", train_val_df,define_model()),target_worker("w2", train_val_df,define_model())])

In [ ]:
tm = tu.topology_manager()

## Define network topology ##

In [ ]:
tm.connect_star(server1, workers.get_all_workers())

In [ ]:
plot = tm.plot_topology()

## Start Training ##

In [ ]:
# initialize server weights as model average
server1.set_init_weights(workers.get_average_weights())

# start training
for epoch in range(epochs):
    print("Epoch: ", epoch)
    new_grad = workers.perform_updates(epoch)
    new_w = server1.aggregate(new_grad)
    workers.set_param(new_w)

## Evaluate Model ##

In [ ]:
# evaluate final model
eval_model = define_model()
eval_model.load_state_dict(new_w)
eval_model.eval()
eval_model = eval_model.cuda()
torch.save(eval_model.state_dict(), "models/experiment2_model.pt")

test_dataset = LFWDataset(data_path, test_df, transform=transforms.Compose([
                                                    transforms.RandomResizedCrop(224),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.ToTensor()
                                                    ]))
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

test_loss, test_acc = perform_evaluation(eval_model, test_dataloader)